## VCF 파일을 DataFrame으로 전환
최종적으로 vcf 파일은 원하는 정보를 추출하기 위해 python의 DataFrame으로 변환하여 사용한다. 여기서는 우리가 vcf 파일을 데이터프레임으로 전환하는데에 있어서 어떠한 전략을 사용할지에 대해서 논한다.

여기서는 python의 vcf 파서를 사용하여 이를 데이터프레임으로 전환한다.

1. Normalization: multi-allelic, Indel left/right alignment
2. Transcript set: RefSeq, UCSC, Ensemble
3. Software tool: ANNOVAR, VEP, SNPEFF 

In [15]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import sys
import vcf
import os
import re
import json
import math

최종적으로 annotation을 마친 vcf 파일을 읽어 들이며 이때 주석부분 (skiprows)은 제외하고 1샘플이 존재함으로 총 10개의 컬럼을 읽어들인다.

In [16]:
ngene_df=pd.read_table("HCT-15_final.vcf", skiprows=494,header=0,usecols=range(10))
vcf_reader=vcf.Reader(open("HCT-15_final.vcf",'r'))
sample_name=vcf_reader.samples[0]

CHROM, POS, ID, REF, ALT, QUAL, FILTER, INFO, FORMAT, 샘플의 10개의 컬럼을 가진 데이터프레임이 생성된다.

In [17]:
ngene_df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HCT-15
0,chr13,32890572,rs1799943;U43746.1:c.-26G>A,G,A,15619.80,.,AB=0.488384;ABP=5.22993;freebayes_AC=1;AFF=0.5...,GT:DP:DPR:RO:QR:AO:QA:GL,"0/1:1894:1894,925:968:25773:925:26830:-1844.59..."
1,chr13,32899388,rs11571610;U43746.1:c.425+67A>C,A,C,5520.29,.,AB=0.479391;ABP=7.12754;freebayes_AC=1;AFF=0.5...,GT:DP:DPR:RO:QR:AO:QA:GL,"0/1:1116:1116,535:581:17398:535:11733:-720.137..."
2,chr13,32906980,rs1801439;U43746.1:c.1365A>G,A,G,3432.77,.,AB=0.498611;ABP=3.02236;freebayes_AC=1;AFF=0.5...,GT:DP:DPR:RO:QR:AO:QA:GL,"0/1:720:720,359:361:10879:359:7266:-437.374,0,..."
3,chr13,32907075,.,C,T,7405.49,.,AB=0.322771;ABP=511.009;freebayes_AC=1;AFF=0.5...,GT:DP:DPR:RO:QR:AO:QA:GL,"0/1:1862:1862,601:1259:38451:601:17029:-972.48..."
4,chr13,32907535,.,CTTTTTTTTTTTG,CTTTTTTTTTG,13160.50,.,AB=0.471519;ABP=16.369;freebayes_AC=1;AFF=0.5;...,GT:DP:DPR:RO:QR:AO:QA:GL,"0/1:1896:1896,894:488:14177:894:22619:-1618.95..."


In [4]:
#for record in vcf_reader:
#    print record.INFO['ANN']

In [5]:
info=[]
ann=[]
for record in vcf_reader:
    #print record.INFO['RSPOS']
    vcf_rec=record.INFO
    vcf_rec['CHROM']=record.CHROM
    vcf_rec['POS']=record.POS
    vcf_rec['ID']=record.ID
    vcf_rec['REF']=record.REF
    vcf_rec['ALT']=record.ALT
    vcf_rec['QUAL']=record.QUAL
    vcf_rec['FILTER']=record.FILTER
    
    vcf_info_ann=','.join(record.INFO['ANN'])
    
    ann.append(vcf_info_ann)
    
    info.append(vcf_rec)

In [6]:
ann_df=pd.DataFrame(ann)
ann_df.columns=['ann']
ann_df
info_c_df=ann_df["ann"].str.split(',').apply(pd.Series,1).stack()

In [7]:
detail_info_df=info_c_df.apply(lambda x: pd.Series(x.split('|')))
detail_info_df.columns=['Allele','Effect','Putative_impact','Gene_name','Gene_id','Feature_type','Feature_id','Tracscript_biotype','Rank_total','HGVS.c','HGVS.p','cDNA_position','CDS_position','Protein_position','Distance_to_feature','Errors']
detail_info_df

Allele                   Effect Putative_impact Gene_name Gene_id  \
0  0            A      5_prime_UTR_variant        MODIFIER     BRCA2   BRCA2   
   1            A    upstream_gene_variant        MODIFIER     ZAR1L   ZAR1L   
1  0            C           intron_variant        MODIFIER     BRCA2   BRCA2   
2  0            G       synonymous_variant             LOW     BRCA2   BRCA2   
3  0            T         missense_variant        MODERATE     BRCA2   BRCA2   
4  0  CTTTTTTTTTG           intron_variant        MODIFIER     BRCA2   BRCA2   
5  0            C       synonymous_variant             LOW     BRCA2   BRCA2   
6  0            G         missense_variant        MODERATE     BRCA2   BRCA2   
7  0            G       synonymous_variant             LOW     BRCA2   BRCA2   
8  0           CT       frameshift_variant            HIGH     BRCA2   BRCA2   
9  0            G       synonymous_variant             LOW     BRCA2   BRCA2   
10 0     CAAAAAAC       frameshift_variant            HIGH     BRCA2   BRCA2   
11 0            C       synonymous_variant             LOW     BRCA2   BRCA2   
12 0           CA           intron_variant        MODIFIER     BRCA2   BRCA2   
13 0            C         missense_variant        MODERATE     BRCA2   BRCA2   
14 0            C           intron_variant        MODIFIER     BRCA2   BRCA2   
15 0            C           intron_variant        MODIFIER     BRCA2   BRCA2   
16 0            A         missense_variant        MODERATE     BRCA2   BRCA2   
17 0            C           intron_variant        MODIFIER     BRCA2   BRCA2   
18 0            C           intron_variant        MODIFIER     BRCA1   BRCA1   
   1            C           intron_variant        MODIFIER     BRCA1   BRCA1   
   2            C           intron_variant        MODIFIER     BRCA1   BRCA1   
   3            C           intron_variant        MODIFIER     BRCA1   BRCA1   
   4            C           intron_variant        MODIFIER     BRCA1   BRCA1   
   5            C           intron_variant        MODIFIER     BRCA1   BRCA1   
19 0            C           intron_variant        MODIFIER     BRCA1   BRCA1   
   1            C           intron_variant        MODIFIER     BRCA1   BRCA1   
   2            C           intron_variant        MODIFIER     BRCA1   BRCA1   
   3            C           intron_variant        MODIFIER     BRCA1   BRCA1   
   4            C           intron_variant        MODIFIER     BRCA1   BRCA1   
...           ...                      ...             ...       ...     ...   
28 0            G       synonymous_variant             LOW     BRCA1   BRCA1   
   1            G       synonymous_variant             LOW     BRCA1   BRCA1   
   2            G       synonymous_variant             LOW     BRCA1   BRCA1   
   3            G           intron_variant        MODIFIER     BRCA1   BRCA1   
   4            G           intron_variant        MODIFIER     BRCA1   BRCA1   
   5            G  non_coding_exon_variant        MODIFIER     BRCA1   BRCA1   
29 0            A       synonymous_variant             LOW     BRCA1   BRCA1   
   1            A       synonymous_variant             LOW     BRCA1   BRCA1   
   2            A       synonymous_variant             LOW     BRCA1   BRCA1   
   3            A           intron_variant        MODIFIER     BRCA1   BRCA1   
   4            A           intron_variant        MODIFIER     BRCA1   BRCA1   
   5            A  non_coding_exon_variant        MODIFIER     BRCA1   BRCA1   
30 0            T         missense_variant        MODERATE     BRCA1   BRCA1   
   1            T         missense_variant        MODERATE     BRCA1   BRCA1   
   2            T         missense_variant        MODERATE     BRCA1   BRCA1   
   3            T           intron_variant        MODIFIER     BRCA1   BRCA1   
   4            T           intron_variant        MODIFIER     BRCA1   BRCA1   
   5            T  non_coding_exon_variant        MODIFIER     BRCA1   BRCA1   
31 0     TA

In [8]:
detail_info_df.index=detail_info_df.index.droplevel(-1)
info_a_df=pd.DataFrame(info)
info_a_df

,1KG_AF,1kg_AC,1kg_AN,AA,AA_AC,AA_GTC,AB,ABP,AC_AFR,AC_AMR,...,exac_AN,freebayes_AC,freebayes_AN,kor2678_Alt,kor2678_AltCnt,kor2678_AltFreq,kor2678_NumSampleAlleles,kor2678_RefCnt,kor2678_RefFreq,technology.illumina
0,0.209265,1048.0,5008.0,G|||,"[452, 3954]","[19, 414, 1770]",0.488384,5.22993,1016.0,2171.0,...,121406.0,1,2,1.0,1005.0,0.37528,2678.0,1673.0,0.62472,1.0
1,0.074281,372.0,5008.0,A|||,NaN,NaN,0.479391,7.12754,NaN,NaN,...,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,0.073682,369.0,5008.0,A|||,"[87, 4315]","[0, 87, 2114]",0.498611,3.02236,236.0,769.0,...,121406.0,1,2,1.0,330.0,0.12323,2678.0,2348.0,0.87677,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,0.322771,511.00900,NaN,NaN,...,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,0.471519,16.36900,NaN,NaN,...,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,0.073482,368.0,5008.0,T|||,"[89, 4317]","[0, 89, 2114]",0.472004,17.72230,246.0,770.0,...,121412.0,1,2,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,0.080072,401.0,5008.0,G|||,"[170, 4236]","[0, 170, 2033]",0.488782,7.10269,425.0,773.0,...,121392.0,1,2,1.0,331.0,0.12360,2678.0,2347.0,0.87640,1.0
7,0.266773,1336.0,5008.0,A|||,"[996, 3410]","[107, 782, 1314]",0.499464,3.01961,2328.0,2405.0,...,121412.0,1,2,1.0,1035.0,0.38648,2678.0,1643.0,0.61352,1.0
8,NaN,NaN,NaN,NaN,NaN,NaN,0.515066,5.10407,NaN,NaN,...,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,0.974042,4878.0,5008.0,G|||,"[4094, 310]","[1902, 290, 10]",0.555769,73.24910,9542.0,11526.0,...,121398.0,1,2,1.0,2678.0,1.00000,2678.0,0.0,0.00000,1.0


In [9]:
#com_df=ngene_df.join(info_a_df)
combine_df=info_a_df.join(detail_info_df)

In [10]:
combine_df

,1KG_AF,1kg_AC,1kg_AN,AA,AA_AC,AA_GTC,AB,ABP,AC_AFR,AC_AMR,...,Feature_id,Tracscript_biotype,Rank_total,HGVS.c,HGVS.p,cDNA_position,CDS_position,Protein_position,Distance_to_feature,Errors
0,0.209265,1048.0,5008.0,G|||,"[452, 3954]","[19, 414, 1770]",0.488384,5.22993,1016.0,2171.0,...,NM_000059.3,protein_coding,2/27,c.-26G>A,,,,,26,
0,0.209265,1048.0,5008.0,G|||,"[452, 3954]","[19, 414, 1770]",0.488384,5.22993,1016.0,2171.0,...,NM_001136571.1,protein_coding,,c.-4510C>T,,,,,4481,
1,0.074281,372.0,5008.0,A|||,NaN,NaN,0.479391,7.12754,NaN,NaN,...,NM_000059.3,protein_coding,4/26,c.425+67A>C,,,,,,
2,0.073682,369.0,5008.0,A|||,"[87, 4315]","[0, 87, 2114]",0.498611,3.02236,236.0,769.0,...,NM_000059.3,protein_coding,10/27,c.1365A>G,p.Ser455Ser,1592/11386,1365/10257,455/3418,,
3,NaN,NaN,NaN,NaN,NaN,NaN,0.322771,511.00900,NaN,NaN,...,NM_000059.3,protein_coding,10/27,c.1460C>T,p.Ala487Val,1687/11386,1460/10257,487/3418,,
4,NaN,NaN,NaN,NaN,NaN,NaN,0.471519,16.36900,NaN,NaN,...,NM_000059.3,protein_coding,10/26,c.1909+21_1909+22delTT,,,,,,
5,0.073482,368.0,5008.0,T|||,"[89, 4317]","[0, 89, 2114]",0.472004,17.72230,246.0,770.0,...,NM_000059.3,protein_coding,11/27,c.2229T>C,p.His743His,2456/11386,2229/10257,743/3418,,
6,0.080072,401.0,5008.0,G|||,"[170, 4236]","[0, 170, 2033]",0.488782,7.10269,425.0,773.0,...,NM_000059.3,protein_coding,11/27,c.2971A>G,p.Asn991Asp,3198/11386,2971/10257,991/3418,,
7,0.266773,1336.0,5008.0,A|||,"[996, 3410]","[107, 782, 1314]",0.499464,3.01961,2328.0,2405.0,...,NM_000059.3,protein_coding,11/27,c.3396A>G,p.Lys1132Lys,3623/11386,3396/10257,1132/3418,,
8,NaN,NaN,NaN,NaN,NaN,NaN,0.515066,5.10407,NaN,NaN,...,NM_000059.3,protein_coding,11/27,c.3599_3600delGT,p.Cys1200fs,3826/11386,3599/10257,1200/3418,,


In [13]:
col_value=list(combine_df.columns.values)
print len(col_value)
print col_value

388
['1KG_AF', '1kg_AC', '1kg_AN', 'AA', 'AA_AC', 'AA_GTC', 'AB', 'ABP', 'AC_AFR', 'AC_AMR', 'AC_Adj', 'AC_CONSANGUINEOUS', 'AC_EAS', 'AC_FEMALE', 'AC_FIN', 'AC_Het', 'AC_Hom', 'AC_MALE', 'AC_NFE', 'AC_OTH', 'AC_POPMAX', 'AC_SAS', 'AFF', 'AFR_AF', 'ALT', 'AMR_AF', 'ANN', 'AN_AFR', 'AN_AMR', 'AN_Adj', 'AN_CONSANGUINEOUS', 'AN_EAS', 'AN_FEMALE', 'AN_FIN', 'AN_MALE', 'AN_NFE', 'AN_OTH', 'AN_POPMAX', 'AN_SAS', 'AO', 'ASP', 'BaseQRankSum', 'CAF', 'CHROM', 'CIGAR', 'CLINVARRS', 'CLNACC', 'CLNALLE', 'CLNDBN', 'CLNDSDB', 'CLNDSDBID', 'CLNHGVS', 'CLNORIGIN', 'CLNREVSTAT', 'CLNSIG', 'CLNSRC', 'CLNSRCID', 'COMMON', 'CSQ', 'ClippingRankSum', 'DB', 'DBSNP', 'DP', 'DPB', 'DPRA', 'DP_HIST', 'EAS_AF', 'EA_AC', 'EA_GTC', 'EPP', 'EPPR', 'ESP_AC', 'ESP_AF_GLOBAL', 'ESP_AF_POPMAX', 'EUR_AF', 'EX_TARGET', 'FILTER', 'FS', 'G5', 'G5A', 'GENEINFO', 'GL', 'GNO', 'GQ_HIST', 'GQ_MEAN', 'GQ_STDDEV', 'GTC', 'GTI', 'GTS', 'HD', 'Het_AFR', 'Het_AMR', 'Het_EAS', 'Het_FIN', 'Het_NFE', 'Het_OTH', 'Het_SAS', 'Hom_AFR', 